# Lab 4

**Submission instructions**

Submit the Colab link to your notebook in Canvas. In addition, take screenshots of the map for each question and submit them to Canvas as well.

In [1]:
import ee
import geemap

In [2]:
geemap.ee_initialize()

## Datasets

The datasets being used in the lab are listed below:

- [TIGER: US Census Counties](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties)
- [Landsat-9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2)
- [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
- [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ)

## Question 1

Write a program to find out how many counties are named `Knox` in the US.

In [5]:
tiger = ee.FeatureCollection("TIGER/2018/Counties")
tiger_df = geemap.ee_to_df(tiger)

In [6]:
tiger_df.head(2)

,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP
0,159915148,11211123,41180,C7,510,00767557,476,F,29510,+38.6356988,-090.2445816,25,,G4020,St. Louis,St. Louis city,29
1,374672862,32584237,16180,C7,510,00863219,456,F,32510,+39.1530600,-119.7473792,00,,G4020,Carson City,Carson City,32


In [10]:
num_knox = tiger_df[tiger_df["NAME"] == "Knox"].shape[0]
print(f"There are {num_knox} counties in the US named Knox.")

There are 9 counties in the US named Knox.


![](https://i.imgur.com/3Jg9P6X.png)

## Question 2

Display Knox county of Tennesse with outline only (no fill color) on the map. (Hint: The `STATEFP` of Tennessee is `47`)

In [16]:
conditions = [ee.Filter.eq('STATEFP', '47'), ee.Filter.eq('NAME', 'Knox')]
knox_tn = tiger.filter(conditions)

In [28]:
m = geemap.Map()
style = {
    'color': 'ff0000ff',
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}
m.addLayer(knox_tn.style(**style))
m.centerObject(knox_tn)

text = "Made by Ilse Paniagua"
m.add_text(text)

m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/28Iaw9b.png)

## Question 3

Use [Landsat-9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2) data to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.

In [52]:
# IP: Note that cloudy pixel metadata name for cloud cover is "CLOUD_COVER"

landsat_collection = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
knox_landsat_image = landsat_collection.filterDate("2022-01-01", "2022-12-31").filterBounds(knox_tn).filterMetadata('CLOUD_COVER', 'less_than', 10).median()

In [99]:
image_viz_params = {
    "bands": ["SR_B4", "SR_B3", "SR_B2"],
    "gamma": [3, 3, 3]

}
p = geemap.Map()
p.addLayer(knox_landsat_image, image_viz_params, "Landsat")
p.centerObject(knox_tn)
p

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/m72amN0.png)

## Question 4

Use [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) data to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.

In [66]:
sentinel_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
knox_sentinel_image = sentinel_collection.filterDate("2022-01-01", "2022-12-31").filterBounds(knox_tn).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10).median()

In [95]:
sentinel_data_viz = {
    "bands": ["B4", "B3", "B2"],
    "gamma": [10, 10, 10]
}

q = geemap.Map()
q.addLayer(knox_sentinel_image, sentinel_data_viz, "Sentinel")
q.centerObject(knox_tn)

q

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/vM1M8Gc.png)

## Question 5

Use [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ) imagery to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.

In [78]:
naip_collection = ee.ImageCollection("USDA/NAIP/DOQQ")
naip_image = naip_collection.filterDate("2021-01-01", "2021-12-31").filterBounds(knox_tn).median()

In [79]:
naip_viz = {
    "bands": ["B", "G", "R"]
}
s = geemap.Map()
s.addLayer(naip_image, naip_viz, "NAIP")
s.centerObject(knox_tn)
s

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/iZSGqGS.png)